<a href="https://colab.research.google.com/github/popuriity/bootcamp_projects/blob/main/nycflights13_dplyr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HOMEWORK - NYCflights13
Five Business Question

## Load library

In [ ]:
# install.packages("nycflights13")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
# Load library
library(nycflights13)
library(tidyverse)

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.4.0      ✔ purrr   0.3.5 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.3      ✔ forcats 0.5.2 

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()





---



---



## 1. Which 5 airlines have the most average departure delay (in minutes).

In [ ]:
flights %>% 
  filter(dep_delay > 0) %>%  # filter only positive delay
  group_by(carrier) %>% 
  summarise(mean_delay = mean(dep_delay)) %>% 
  arrange(desc(mean_delay)) %>% 
  left_join(airlines, by = "carrier") %>% 
  select(airline = name,
         code = carrier,
         mean_delay_mins = mean_delay ) %>% 
  head(5)

airline,code,mean_delay_mins
<chr>,<chr>,<dbl>
SkyWest Airlines Inc.,OO,58.00000
Mesa Airlines Inc.,YV,52.95279
ExpressJet Airlines Inc.,EV,50.32979
Endeavor Air Inc.,9E,48.92001
Frontier Airlines Inc.,F9,45.13783


SkyWest, Mesa, ExpressJet, Endeavor, Frontier Airlines are top5 of deley departure in 2013, respectively.



---



---



## 2. The longest route from each airports in NYC (Newark Liberty Intl, John F Kennedy Intl, La Guardia).

In [ ]:
flights %>%
  group_by(origin, dest) %>%
  summarise(distance = max(distance)) %>% 
  top_n(1,distance) %>% 
  inner_join(airports, by = c("origin"="faa")) %>% 
  select(1:3, name_origin = name) %>% 
  inner_join(airports, by = c("dest"="faa")) %>% 
  select(1:4, name_dest = name)

`summarise()` has grouped output by 'origin'. You can override using the
`.groups` argument.


origin,dest,distance,name_origin,name_dest
<chr>,<chr>,<dbl>,<chr>,<chr>
EWR,HNL,4963,Newark Liberty Intl,Honolulu Intl
JFK,HNL,4983,John F Kennedy Intl,Honolulu Intl
LGA,DEN,1620,La Guardia,Denver Intl


*   Newark to Honolulu 4963 miles
*   JFK to Honolulu 4983 miles
*   La Guardia to Denver 1620 miles


---



## 3. Top10 of aircraft manufacturer that operated flight in 2013

In [ ]:
flights %>% 
  left_join(select(planes, tailnum, manufacturer), by = "tailnum") %>% 
  group_by(manufacturer) %>% 
  summarise(n=n()) %>% 
  mutate(manufacturer = if_else(is.na(manufacturer), "Others", manufacturer) ,
         percentage = n*100/sum(n)) %>% 
  arrange(desc(n)) %>% 
  head(10)

manufacturer,n,percentage
<chr>,<int>,<dbl>
BOEING,82912,24.6193315
EMBRAER,66068,19.6177875
Others,52606,15.6204718
AIRBUS,47302,14.0455377
AIRBUS INDUSTRIE,40891,12.1418985
BOMBARDIER INC,28272,8.3948975
MCDONNELL DOUGLAS AIRCRAFT CO,8932,2.6522080
MCDONNELL DOUGLAS,3998,1.1871392
CANADAIR,1594,0.4733116




---



---



## 4. The most popular route from each airport on Christmas-New year period. (24 Dec - 31 Dec)

In [ ]:
flights %>% 
  filter(month == 12) %>% 
  filter(between(day,24,31) ) %>% 
  group_by(origin, dest) %>% 
  summarise(n=n()) %>% 
  top_n(1, n)  %>% 
  inner_join(airports, by = c("origin"="faa")) %>% 
  select(1:3, name_origin = name) %>% 
  inner_join(airports, by = c("dest"="faa")) %>% 
  select(1:4, name_dest = name)

`summarise()` has grouped output by 'origin'. You can override using the
`.groups` argument.


origin,dest,n,name_origin,name_dest
<chr>,<chr>,<int>,<chr>,<chr>
EWR,MCO,127,Newark Liberty Intl,Orlando Intl
JFK,LAX,231,John F Kennedy Intl,Los Angeles Intl
LGA,ATL,206,La Guardia,Hartsfield Jackson Atlanta Intl


The popular route in Chirstmas to New Year period of Newark is Orlando, JFK is LA and La Guardia is Atlanta.



---



---



## 5. Proportion between delay and early arrival flight in 2013.

In [ ]:
flights %>% 
  mutate( arrival_type = case_when(arr_delay > 0 ~ "delay",
                                    arr_delay < 0 ~ "early",
                                    arr_delay == 0 ~ "on time")) %>%
  count(arrival_type) %>% 
  mutate(percent = n*100/sum(n)) %>% 
  arrange(desc(percent))

arrival_type,n,percent
<chr>,<int>,<dbl>
early,188933,56.100494
delay,133004,39.493313
NA,9430,2.800081
on time,5409,1.606112


56.10% of flights from NYC in 2013 arrived at destination early, while 39.49% are lated. On-time arrival flights is 1.61% and NA is 2.80%